https://colab.research.google.com/drive/1UNkN-T11SoE0fBkF0-nWmgl90TNbkN0e?usp=sharing#scrollTo=_JhI5MBhXHiM
# Choropleths
The file http://datasci112.stanford.edu/data/kidney_cancer_by_county.csv contains data on kidney cancer deaths by county in the 1980s. (Note that Alaska is not in this data.)

Let's make a choropleth showing the kidney cancer death rate by county. We will first need to divide cancer deaths by population to get the rate.

In [1]:
import pandas as pd
df_cancer = pd.read_csv("http://datasci112.stanford.edu/data/kidney_cancer_by_county.csv")
df_cancer

county   fips  deaths  population
0        Autauga County, Alabama   1001       3       63418
1        Baldwin County, Alabama   1003      22      183099
2        Barbour County, Alabama   1005       1       33652
3           Bibb County, Alabama   1007       1       30664
4         Blount County, Alabama   1009       8       89944
...                          ...    ...     ...         ...
3105  Sweetwater County, Wyoming  56037       3      108207
3106       Teton County, Wyoming  56039       0       26530
3107       Uinta County, Wyoming  56041       2       50503
3108    Washakie County, Wyoming  56043       2       23888
3109      Weston County, Wyoming  56045       1       18388

[3110 rows x 4 columns]

In [2]:
df_cancer["cancer_death_rate"] = (df_cancer["deaths"] /
                                  df_cancer["population"])
df_cancer

county   fips  deaths  population  cancer_death_rate
0        Autauga County, Alabama   1001       3       63418           0.000047
1        Baldwin County, Alabama   1003      22      183099           0.000120
2        Barbour County, Alabama   1005       1       33652           0.000030
3           Bibb County, Alabama   1007       1       30664           0.000033
4         Blount County, Alabama   1009       8       89944           0.000089
...                          ...    ...     ...         ...                ...
3105  Sweetwater County, Wyoming  56037       3      108207           0.000028
3106       Teton County, Wyoming  56039       0       26530           0.000000
3107       Uinta County, Wyoming  56041       2       50503           0.000040
3108    Washakie County, Wyoming  56043       2       23888           0.000084
3109      Weston County, Wyoming  56045       1       18388           0.000054

[3110 rows x 5 columns]

# Shapefiles
First, we need a shapefile of the counties of the U.S. Let's download it from the U.S. Census Bureau website. Remember that our data above is from the 1980s. So we should choose a shapefile from around that time. Let's download the 1990 shapefile.

Next, we upload the .zip file to the Colab runtime and read in the shapefile below.

In [3]:
import geopandas as gpd

# Make sure you have uploaded the .zip file from the Census Bureau to Colab.
gdf_counties = gpd.read_file("zip://co99_d90_shp.zip")
gdf_counties

C:\Users\fatma\anaconda3\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


DataSourceError: '/vsizip/co99_d90_shp.zip' does not exist in the file system, and is not recognized as a supported dataset name.

# Joining the Data
To make a choropleth, we will need a GeoDataFrame that contains both the geometries and the cancer death rates. This means that we have to join gdf_counties to df_cancer. But what key do we join on?

The FIPS code is a federal standard for identifying places. Counties are identified by a 5-digit code consisting of a 2-digit state code and a 3-digit county code.

The cancer data has the 5-digit county FIPS code as an integer. The shapefile has the state and county FIPS codes in separate columns. So let's combine them into a single FIPS code.

In [4]:
gdf_counties["fips"] = (
    gdf_counties["ST"].str.lstrip("0") +
    gdf_counties["CO"]
).astype(int)
gdf_counties

NameError: name 'gdf_counties' is not defined

Now, we can join gdf_counties to df_cancer on this combined FIPS code.

In [5]:
gdf_counties_cancer = gdf_counties.merge(
    df_cancer,
    on="fips",
    how="outer"  # outer join will help us detect counties that failed to join
    )
gdf_counties_cancer

NameError: name 'gdf_counties' is not defined

Let's look for missing values.

In [6]:
# in `df_cancer` but not in `gdf_counties`
gdf_counties_cancer[gdf_counties_cancer["geometry"].isnull()]

NameError: name 'gdf_counties_cancer' is not defined

In [7]:
# in `gdf_counties` but not in `df_cancer`
gdf_counties_cancer[gdf_counties_cancer["cancer_death_rate"].isnull()]

NameError: name 'gdf_counties_cancer' is not defined

# Making the Choropleth
Now that we have joined the cancer data to the geographic data, making the choropleth is easy. We simply pass the column= argument to .plot().

In [8]:
gdf_counties_cancer.plot(column="cancer_death_rate")

NameError: name 'gdf_counties_cancer' is not defined

In [9]:
Let's make the figure bigger, and set the axes limits to focus on the continental U.S.

SyntaxError: unterminated string literal (detected at line 1) (1069898873.py, line 1)

In [10]:
ax = gdf_counties_cancer.plot(
    column="cancer_death_rate",
    figsize=(16, 8))

# Focus on the continental U.S.
ax.set_xlim(-126, -66)
ax.set_ylim(24, 50)

NameError: name 'gdf_counties_cancer' is not defined

What counties have the highest cancer rates? Let's highlight the counties in the top 10% of the cancer rates in red.

In [11]:
gdf_counties_cancer["to_highlight"] = (
    gdf_counties_cancer["cancer_death_rate"] >=
    gdf_counties_cancer["cancer_death_rate"].quantile(0.90))

ax = gdf_counties_cancer.plot(
    column="to_highlight",
    figsize=(16, 8),
    cmap="Reds")
ax.set_xlim(-126, -66)
ax.set_ylim(24, 50)

NameError: name 'gdf_counties_cancer' is not defined

Why do you think the counties with the highest kidney cancer death rates are all in the middle of the country?